# Feature Selection

Feature engineering is one of the most important task in any machine learning project. Feature selection is one of the subtask that is carried out to select the features which best represent the target variable. There are various methods for feature selection that will be using the project including feature selection using correlation, select k best using chi square , select from models such as logistic regression, random forest etc.
Feature selection can be:
1. Filter-based
2. Wrapper-based
3. Embedded methods

In [ ]:
# Load the packages

import pandas as pd
from sklearn.feature_selection import SelectKBest, RFECV, SequentialFeatureSelector, SelectFromModel
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression